### 处理思路：遍历所有的经纬度，按照经纬度分别读取对应的种植开始时间和收获开始时间，以确定成长期对应的月份，并将其存储到对应的网格中。

In [1]:
import pandas as pd
import numpy as np
import xarray as xr
import time
import os

In [2]:
def get_filenames(format):
    # 获取当前路径下的所有.nc文件名
    current_path = os.getcwd()
    file_names = [file for file in os.listdir(current_path) if file.endswith(format)]

    return(file_names)

In [3]:
# 获取农作物成长期的起始时间和结束时间
def get_crop_growth_period(filename):

    # 读取平均温度
    # filename = 'Rice.crop.calendar.fill.nc'
    # 打开 NetCDF 文件，返回 Dataset 对象
    with xr.open_dataset(filename, engine='netcdf4', decode_times=False) as ds:
        latitude_start = ds['plant.start'].latitude.values
        longitude_start = ds['plant.start'].longitude.values
        
        latitude_end = ds['harvest.start'].latitude.values
        longitude_end = ds['harvest.start'].longitude.values
        
        lats, lons, values_start, values_end = [], [], [], []
        
        for j in range(len(latitude_start)):
            for k in range(len(longitude_start)):
                value_start = ds['plant.start'].sel(
                    longitude=longitude_start[k],  # 指定目标经度
                    latitude=latitude_start[j],    # 指定目标纬度
                    method='nearest'  # 若没有精确匹配，取最近的点
                )
        
                value_end = ds['harvest.start'].sel(
                    longitude=longitude_end[k],  # 指定目标经度
                    latitude=latitude_end[j],    # 指定目标纬度
                    method='nearest'  # 若没有精确匹配，取最近的点
                )
                
                lats.append(latitude_start[j])
                lons.append(longitude_start[k])
                values_start.append(value_start.values)
                values_end.append(value_end.values)

    return(lats, lons, values_start, values_end)

In [4]:
# 判断农作物成长期对应的月份
def judge_month(days):

    if not np.isnan(days):
        month_days = [31,30,31,30,31,30,31,31,30,31,30,31]
        cumulative = np.cumsum(month_days)
        k = np.argmax(cumulative >= days)
        return(k+1)
    else:
        return(np.nan)

In [5]:
# 获取农作物成长期对应的月份
def get_month(datas):

    # return [judge_month(day) for day in datas]
    return np.vectorize(judge_month)(datas)

In [6]:
# 创建经纬度的网格码对照表(360x720)函数
def cread_grids_table(m,n):

    grids_table = np.array(list(range(1, m*n+1))).reshape(m,n)
    grids_table = pd.DataFrame(grids_table, columns=range(1,n+1), index=range(1,m+1))
    return(grids_table)

In [7]:
# 查找经纬度对应的网格码
def search_grids_num(lat,lon,grids_table):
    
    m = int((90+lat)*2+1)
    n = int((180+lon)*2+1)

    return(grids_table.loc[m,n])

In [8]:
# 按照网格码表格生成df数据集的网格化结果
def clear_data(df, grids_table):
    
    grids_num = []
    df_result = pd.DataFrame()
    
    for k in df.index:
        grids_num.append(search_grids_num(df['lat'].values[k],df['lon'].values[k], grids_table)) #计算经纬度所属的网格码
    
    df['grid_num'] = grids_num
    
    # 按 grid_num 分组，对 clay、BD、SOC、pH、CEC 列求平均值
    df_result = df.groupby('grid_num')[df.columns[2:-1].tolist()].mean().round(2).reset_index()
    
    return(df_result)

In [9]:
if __name__ == "__main__":

    # 创建网格码对照表实例
    m = int(180/0.5)
    n = int(360/0.5)
    
    grids_table = cread_grids_table(m,n)

    #获取当前路径下的所有.nc文件
    format = '.nc'
    all_filenames = get_filenames(format)

    # 获取文件数据
    for filename in all_filenames:
        
        # 记录开始时间
        start_time = time.time()  # 返回当前时间的时间戳（秒）
        
        # 获取文件数据
        lats, lons, starts, ends = get_crop_growth_period(filename)
    
        # 计算弄作文生长季的起始和截止月份并合并数据
        start_month = get_month(starts)
        end_month = get_month(ends)
        df = pd.DataFrame({'lat':lats, 'lon':lons, 'start':starts, 'end':ends, 'start_month':start_month, 'end_month':end_month})
        
        # 计算网格化结果
        df_result = clear_data(df, grids_table)
        
        # 存储结果
        df_result.to_csv(filename.rsplit('.',1)[0]+'(grids).csv', index=False)

        # 记录结束时间
        end_time = time.time()
        
        # 计算耗时
        elapsed_time = end_time - start_time
        print(f"耗时: {elapsed_time/60:.2f} 分")

耗时: 2.01 分
耗时: 2.01 分
耗时: 1.99 分
耗时: 1.99 分
耗时: 1.98 分
耗时: 1.99 分
